**JsonOutputParser를 사용하면 JSON 형태의 출력도 가능하다.**

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

question = "지구 온난화의 심각성 대해 알려주세요."

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = prompt | model | parser  # 체인을 구성합니다.

output = chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행


In [3]:
output

{'description': '지구 온난화는 지구의 기온이 상승함에 따라 발생하는 현상으로, 인간 활동에 의해 온실가스가 대기 중에 증가함으로써 가속화되고 있습니다. 이로 인해 극지방의 빙하와 빙하가 녹아 해수면이 상승하고, 극지방의 생태계가 변화하며 자연재해의 빈도와 강도가 증가하는 등 지구 전반에 부정적인 영향을 미치고 있습니다.',
 'hashtags': '지구온난화, 기후변화'}